
# Residual Blok Rehberi (Sıfırdan İleri Seviyeye)

Bu notlar, **residual (skip) bağlantının** mantığını en temelden başlayıp pratik entegrasyona kadar götürür.  
Amaç: Bir CNN modelini adım adım büyütürken **residual blokları** doğru tanımlamak ve **mevcut bir mimariye** temiz şekilde entegre edebilmek.

> Kısa sözlük  
- **F(x)**: Bloğun öğrenmeye çalıştığı dönüşüm (conv/BN/act/attention vb.)  
- **skip(x)**: Kimlik (identity) ya da projeksiyon (1×1 conv) yoluyla taşınan bilgi  
- **Çıkış**: `y = F(x) + skip(x)`



## 1) Residual bağlantı neyi çözer?

Derin ağlarda iki klasik problem ortaya çıkar:

1. **Optimizasyon zorlaşır** (derinlik arttıkça öğrenme “takılabilir”).  
2. **Gradyan akışı zayıflayabilir** (çok katmandan geri akış zorlaşır).

Residual bağlantı şunu garanti eder:

- Eğer `F(x)` başlangıçta işe yaramıyorsa bile model en azından **skip(x)** üzerinden “akabilir”.  
- Eğitim ilerledikçe `F(x)` “düzeltme” (residual) öğrenir.

Bu yüzden residual bloklar pratikte şu sezgiyle çalışır:

> “Tamamen yeni bir şey üretmek yerine, **mevcut temeli** küçük düzeltmelerle iyileştir.”



## 2) Residual blok yazarken dikkat edilecek 4 şey (Checklist)

Aşağıdaki dört maddeyi kontrol ettiğinde residual blokların %90’ı düzgün çalışır:

1. **Şekil uyumu (shape)**  
   - `F(x)` ve `skip(x)` toplanacağı için `(B, C, H, W)` boyutlarının uyumlu olması gerekir.

2. **Downsample / stride**  
   - Stride=2 gibi bir durum varsa `skip(x)` yolu da aynı şekilde downsample edilmelidir (genelde 1×1 conv + stride).

3. **Kanal geçişi (in_ch → out_ch)**  
   - Kanal sayısı değişiyorsa yine 1×1 projeksiyon gerekir.

4. **Norm/Activation sırası (Pre-Act vs Post-Act)**  
   - Post-Act (klasik): `Conv → BN → ReLU`  
   - Pre-Act (v2): `BN → ReLU → Conv`


In [1]:

# Ortam: PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

def count_params(m: nn.Module) -> int:
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

def demo_shapes(m: nn.Module, x: torch.Tensor, name: str = ""):
    with torch.no_grad():
        y = m(x)
    print(f"{name:>18}  in: {tuple(x.shape)} -> out: {tuple(y.shape)} | params: {count_params(m):,}")



## 3) En basit blok: Conv-BN-Act

Residual’a geçmeden önce temel bir yapı taşı kuralım: `Conv + BN + Activation`.


In [ ]:

class ConvBNAct(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=None, act="relu"):
        super().__init__()
        if p is None:
            p = k // 2
        self.conv = nn.Conv2d(cin, cout, k, stride=s, padding=p, bias=False)
        self.bn = nn.BatchNorm2d(cout)
        if act == "relu":
            self.act = nn.ReLU(inplace=True)
        elif act == "silu":
            self.act = nn.SiLU(inplace=True)
        else:
            raise ValueError("act: 'relu' veya 'silu' destekleniyor.")
    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

x = torch.randn(2, 3, 32, 32)
demo_shapes(ConvBNAct(3, 16), x, "ConvBNAct")


         ConvBNAct  in: (2, 3, 32, 32) -> out: (2, 16, 32, 32) | params: 464



## 4) Post-Activation Basic Residual Block (ResNet v1 tarzı)

Klasik şekil:

- Residual yol (F): `Conv → BN → ReLU → Conv → BN`
- Skip yol: `x` ya da `1×1 conv` (boyut uyumu için)
- Çıkış: `ReLU(F(x) + skip(x))`

> Not: Bazı implementasyonlarda en sondaki ReLU kaldırılabilir, ancak v1’de yaygın olan hali aşağıdaki gibidir.


In [3]:

class BasicBlock_PostAct(nn.Module):
    def __init__(self, cin, cout, stride=1):
        super().__init__()
        self.conv1 = ConvBNAct(cin, cout, k=3, s=stride, act="relu")
        self.conv2 = nn.Sequential(
            nn.Conv2d(cout, cout, 3, padding=1, bias=False),
            nn.BatchNorm2d(cout),
        )
        self.proj = None
        if stride != 1 or cin != cout:
            self.proj = nn.Sequential(
                nn.Conv2d(cin, cout, 1, stride=stride, bias=False),
                nn.BatchNorm2d(cout),
            )
        self.out_act = nn.ReLU(inplace=True)

    def forward(self, x):
        skip = x if self.proj is None else self.proj(x)
        y = self.conv1(x)
        y = self.conv2(y)
        y = y + skip
        return self.out_act(y)

demo_shapes(BasicBlock_PostAct(16, 16), torch.randn(2,16,32,32), "BasicPostAct s1")
demo_shapes(BasicBlock_PostAct(16, 32, stride=2), torch.randn(2,16,32,32), "BasicPostAct s2")


   BasicPostAct s1  in: (2, 16, 32, 32) -> out: (2, 16, 32, 32) | params: 4,672
   BasicPostAct s2  in: (2, 16, 32, 32) -> out: (2, 32, 16, 16) | params: 14,528



## 5) Pre-Activation Basic Residual Block (ResNet v2 tarzı)

Buradaki kritik fark: **BN+ReLU, conv’dan önce** gelir.

- Residual yol (F): `BN → ReLU → Conv → BN → ReLU → Conv`
- Skip yol: `x` ya da `1×1 conv` (boyut uyumu için)
- Çıkış: genelde `F(x) + skip(x)` (en sonda ekstra ReLU şart değil)

Bu tasarım, özellikle derin ağlarda optimizasyonu kolaylaştırmasıyla bilinir.


In [4]:

class BasicBlock_PreAct(nn.Module):
    def __init__(self, cin, cout, stride=1):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(cin)
        self.conv1 = nn.Conv2d(cin, cout, 3, stride=stride, padding=1, bias=False)

        self.bn2 = nn.BatchNorm2d(cout)
        self.conv2 = nn.Conv2d(cout, cout, 3, padding=1, bias=False)

        self.proj = None
        if stride != 1 or cin != cout:
            self.proj = nn.Conv2d(cin, cout, 1, stride=stride, bias=False)

    def forward(self, x):
        skip = x if self.proj is None else self.proj(x)

        y = F.relu(self.bn1(x), inplace=True)
        y = self.conv1(y)

        y = F.relu(self.bn2(y), inplace=True)
        y = self.conv2(y)

        return y + skip

demo_shapes(BasicBlock_PreAct(16, 16), torch.randn(2,16,32,32), "BasicPreAct s1")
demo_shapes(BasicBlock_PreAct(16, 32, stride=2), torch.randn(2,16,32,32), "BasicPreAct s2")


    BasicPreAct s1  in: (2, 16, 32, 32) -> out: (2, 16, 32, 32) | params: 4,672
    BasicPreAct s2  in: (2, 16, 32, 32) -> out: (2, 32, 16, 16) | params: 14,432



## 6) “Wide” fikri nerede? (Widen Factor k)

Önemli nokta:

- **Wide** olmak, bloğun “özel bir forward yapması” değildir.  
- Wide olmak, bloğun **kanal planıyla** ilgilidir.

Yani `out_ch` değerlerini `k` ile çarparak stage’leri baştan geniş kurarsın:
- stage1: `16k`
- stage2: `32k`
- stage3: `64k`

Bu yüzden `BasicBlock_PreAct` aynı kalıp, sadece `cout` değerleri büyür.


In [5]:

def make_stage(block_cls, cin, cout, n_blocks, first_stride):
    layers = [block_cls(cin, cout, stride=first_stride)]
    for _ in range(1, n_blocks):
        layers.append(block_cls(cout, cout, stride=1))
    return nn.Sequential(*layers)

class TinyWideResNet(nn.Module):
    def __init__(self, in_ch=3, num_classes=10, depth=28, k=2):
        super().__init__()
        assert (depth - 4) % 6 == 0
        n = (depth - 4) // 6
        base = 16
        ch1, ch2, ch3 = base*k, base*2*k, base*4*k

        self.stem = nn.Conv2d(in_ch, base, 3, padding=1, bias=False)

        # Pre-Act blok kullanıyoruz
        self.stage1 = make_stage(BasicBlock_PreAct, base, ch1, n, first_stride=1)
        self.stage2 = make_stage(BasicBlock_PreAct, ch1,  ch2, n, first_stride=2)
        self.stage3 = make_stage(BasicBlock_PreAct, ch2,  ch3, n, first_stride=2)

        self.bn = nn.BatchNorm2d(ch3)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(ch3, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = F.relu(self.bn(x), inplace=True)
        x = self.pool(x).flatten(1)
        return self.fc(x)

m = TinyWideResNet(depth=28, k=2)
demo_shapes(m, torch.randn(2,3,32,32), "TinyWideResNet")


    TinyWideResNet  in: (2, 3, 32, 32) -> out: (2, 10) | params: 1,467,610



## 7) Bottleneck (Expansion) fikri

Bottleneck bloklar, hesap maliyetini kontrol ederek daha “geniş” temsile izin verir.

Klasik bottleneck akışı (ResNet-50 tarzı):
- 1×1 (daralt)
- 3×3 (işle)
- 1×1 (genişlet / expansion)

`expansion` çoğunlukla 4’tür: `out = bottleneck_ch * 4`.


In [6]:

class Bottleneck_PostAct(nn.Module):
    expansion = 4
    def __init__(self, cin, bottleneck_ch, stride=1):
        super().__init__()
        out_ch = bottleneck_ch * self.expansion
        self.conv1 = ConvBNAct(cin, bottleneck_ch, k=1, s=1, p=0, act="relu")
        self.conv2 = ConvBNAct(bottleneck_ch, bottleneck_ch, k=3, s=stride, p=1, act="relu")
        self.conv3 = nn.Sequential(
            nn.Conv2d(bottleneck_ch, out_ch, 1, bias=False),
            nn.BatchNorm2d(out_ch),
        )
        self.proj = None
        if stride != 1 or cin != out_ch:
            self.proj = nn.Sequential(
                nn.Conv2d(cin, out_ch, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch),
            )
        self.out_act = nn.ReLU(inplace=True)

    def forward(self, x):
        skip = x if self.proj is None else self.proj(x)
        y = self.conv1(x)
        y = self.conv2(y)
        y = self.conv3(y)
        y = y + skip
        return self.out_act(y)

demo_shapes(Bottleneck_PostAct(64, 64, stride=1), torch.randn(2,64,32,32), "Bottleneck s1")
demo_shapes(Bottleneck_PostAct(64, 64, stride=2), torch.randn(2,64,32,32), "Bottleneck s2")


     Bottleneck s1  in: (2, 64, 32, 32) -> out: (2, 256, 32, 32) | params: 75,008
     Bottleneck s2  in: (2, 64, 32, 32) -> out: (2, 256, 16, 16) | params: 75,008



## 8) SE (Squeeze-Excitation) modülü

SE, **channel attention** türüdür:

1. Squeeze: `GlobalAvgPool` ile her kanal için tek bir özet çıkarılır.  
2. Excite: Küçük bir MLP (genelde iki FC/1×1 conv) ile kanal önemleri öğrenilir.  
3. Sigmoid: ağırlıklar `[0,1]` aralığına çekilir.  
4. Ölçekleme: `x * w`

> SE tek başına residual değildir.  
> Residual bloğun içindeki `F(x)` dönüşümüne eklenen bir modüldür.


In [7]:

class SEBlock(nn.Module):
    def __init__(self, channels: int, reduction: int = 16, min_hidden: int = 4):
        super().__init__()
        hidden = max(min_hidden, channels // reduction)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(channels, hidden, 1, bias=True),
            nn.SiLU(inplace=True),
            nn.Conv2d(hidden, channels, 1, bias=True),
            nn.Sigmoid(),
        )

    def forward(self, x):
        w = self.fc(self.pool(x))
        return x * w

# test
demo_shapes(SEBlock(64), torch.randn(2,64,16,16), "SEBlock")


           SEBlock  in: (2, 64, 16, 16) -> out: (2, 64, 16, 16) | params: 580



## 9) SE-Residual Block (en yaygın yerleşim)

Pratikte en sık görülen yerleşim:

- `F(x)` dönüşümü tamamlanır (conv/BN/act/conv/BN)  
- **SE, F(x) çıktısına uygulanır**  
- sonra `+ skip` yapılır  
- en sona aktivasyon (post-act) opsiyoneldir

Aşağıdaki örnek: Post-Act tarzı bir basic blok + SE.


In [8]:

class SEResBlock(nn.Module):
    def __init__(self, cin, cout, stride=1, reduction=16):
        super().__init__()
        self.conv1 = ConvBNAct(cin, cout, k=3, s=stride, p=1, act="silu")
        self.conv2 = nn.Sequential(
            nn.Conv2d(cout, cout, 3, padding=1, bias=False),
            nn.BatchNorm2d(cout),
        )
        self.se = SEBlock(cout, reduction=reduction)

        self.proj = None
        if stride != 1 or cin != cout:
            self.proj = nn.Sequential(
                nn.Conv2d(cin, cout, 1, stride=stride, bias=False),
                nn.BatchNorm2d(cout),
            )
        self.out_act = nn.SiLU(inplace=True)

    def forward(self, x):
        skip = x if self.proj is None else self.proj(x)
        y = self.conv1(x)
        y = self.conv2(y)
        y = self.se(y)
        y = y + skip
        return self.out_act(y)

demo_shapes(SEResBlock(32,32), torch.randn(2,32,32,32), "SEResBlock s1")
demo_shapes(SEResBlock(32,64,stride=2), torch.randn(2,32,32,32), "SEResBlock s2")


     SEResBlock s1  in: (2, 32, 32, 32) -> out: (2, 32, 32, 32) | params: 18,852
     SEResBlock s2  in: (2, 32, 32, 32) -> out: (2, 64, 16, 16) | params: 58,308



## 10) Bir modeli basitten ileriye doğru kurma (progressive build)

Aşağıdaki sıra, pratikte en anlaşılır “büyütme” yoludur:

1. **Baseline CNN**: ConvBNAct + pooling + classifier  
2. **Residual CNN**: BasicBlock (post veya pre) ile stage’ler  
3. **Wide**: kanal planını `k` ile ölçekle (WideResNet mantığı)  
4. **Bottleneck**: büyük modellerde hesap/parametre verimliliği  
5. **SE (veya başka attention)**: seçicilik, kanal önemlendirme  
6. **Ek düzenlemeler**: DropPath, GN/LN, FPN/PAN (task’e göre)

Aşağıda 1→2 geçişini “mini” bir örnekle gösteriyoruz.


In [9]:

class BaselineCNN(nn.Module):
    def __init__(self, num_classes=10, in_ch=3, base=32):
        super().__init__()
        self.features = nn.Sequential(
            ConvBNAct(in_ch, base, 3, 1, 1, act="relu"),
            ConvBNAct(base, base, 3, 1, 1, act="relu"),
            nn.MaxPool2d(2),  # 32->16
            ConvBNAct(base, base*2, 3, 1, 1, act="relu"),
            nn.MaxPool2d(2),  # 16->8
            ConvBNAct(base*2, base*4, 3, 1, 1, act="relu"),
        )
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(base*4, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x).flatten(1)
        return self.fc(x)

demo_shapes(BaselineCNN(), torch.randn(2,3,32,32), "BaselineCNN")


       BaselineCNN  in: (2, 3, 32, 32) -> out: (2, 10) | params: 104,042



## 11) Entegrasyon mantığı: “blok aynı, yerleştirme değişir”

Bir residual/attention bloğu genelde **şu nedenle taşınabilir**:

- Girdi/çıktı formatı: `(B, C, H, W)` → `(B, C', H', W')`  
- Blok “yerel” bir dönüşüm yapar  
- Toplama yapılacaksa `skip(x)` boyut uyumunu garanti eder

**Sınıflandırma**: backbone + pooling + fc  
**Detection (YOLO vb.)**: backbone + neck (FPN/PAN) + head  
**Segmentation**: encoder + decoder (+ skip-merge) + mask head

Entegrasyonda değişen şey çoğu zaman blok değil, blokların **hangi stage’e** yerleştirildiğidir.



## 12) Mini kontrol: “Bu blok entegre olur mu?”

Bir bloğu bir modele eklemeden önce şu 3 test yeterlidir:

1. **Shape testi**: Rastgele `x` ile `forward` çalışıyor mu?  
2. **Downsample testi**: stride=2 senaryosu çalışıyor mu?  
3. **Grad testi**: `loss = y.mean()` ile `backward()` patlıyor mu?

Aşağıdaki hücre bu üç kontrolü hızlıca yapar.


In [ ]:
def quick_checks(block: nn.Module, x: torch.Tensor):
    y = block(x)          # shape
    y.mean().backward()   # grad
    return y

blk = SEResBlock(16, 32, stride=2)
x = torch.randn(2, 16, 32, 32, requires_grad=True)
y = quick_checks(blk, x)
print("ok:", tuple(y.shape))


ok: (2, 32, 16, 16)



## 13) Kapanış

Bu notlardan sonra şu fark netleşmiş olmalı:

- **Residual**: “bağlantı / öğrenme çerçevesi”  
- **Attention (SE vb.)**: `F(x)` içinde “seçicilik”  
- **Wide / Grouped / Bottleneck**: `F(x)` içindeki **kanal/hesap tasarımı**  
- **Pre/Post-Act**: blok içindeki **sıralama şablonu**

İleride bir blok gördüğünde yapılacak en pratik şey:
1) `skip` yolu ne? (identity mi proj mi)  
2) `F(x)` hangi sırayla? (pre mi post mu)  
3) toplama nerede? (önce/sonra activation var mı)
